# LLM Sentiment Analysis Research Notebook

This notebook demonstrates the UPGRADE-014 LLM sentiment integration features:

## Contents
1. LLM Ensemble Setup (FinBERT + GPT-4o + Claude)
2. Sentiment Analysis Examples
3. Sentiment Filter Usage
4. News Alert Manager
5. LLM Guardrails
6. Dynamic Weight Adjustment
7. Sentiment Persistence
8. Integration with Trading Decisions

**UPGRADE-014**: LLM Sentiment Integration (December 2025)

Based on TradingAgents multi-agent framework research for financial applications.

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# LLM Sentiment Integration (UPGRADE-014)
from llm import (
    # Base classes
    Sentiment,
    SentimentResult,
    # Sentiment analyzers
    SimpleSentimentAnalyzer,
    create_sentiment_analyzer,
    # Ensemble
    LLMEnsemble,
    EnsembleResult,
    create_ensemble,
    # Sentiment Filter (UPGRADE-014)
    SentimentFilter,
    SentimentSignal,
    FilterResult,
    FilterDecision,
    FilterReason,
    create_sentiment_filter,
    create_signal_from_ensemble,
    # News Alert Manager (UPGRADE-014)
    NewsAlertManager,
    NewsEvent,
    NewsImpact,
    NewsEventType,
    NewsAlertConfig,
    create_news_alert_manager,
    # LLM Guardrails (UPGRADE-014)
    LLMGuardrails,
    TradingConstraints,
    GuardrailCheckResult,
    create_llm_guardrails,
)

print('LLM Sentiment modules imported successfully')

## 1. LLM Ensemble Setup

The ensemble combines multiple LLM providers for robust sentiment analysis:
- **FinBERT**: Financial-specific BERT model (40% weight)
- **GPT-4o**: OpenAI's latest model (30% weight)
- **Claude**: Anthropic's model (30% weight)

Weights are dynamically adjusted based on historical accuracy.

In [ ]:
# Create a simple sentiment analyzer for demonstration
# (In production, use create_ensemble with API keys)
analyzer = SimpleSentimentAnalyzer()

# Example news headlines
headlines = [
    "Apple reports record quarterly earnings, beats analyst expectations",
    "Fed signals potential rate hike amid inflation concerns",
    "Tesla recalls 500,000 vehicles due to safety issues",
    "Microsoft acquires gaming company in $68B deal",
    "Bankruptcy fears grow as company misses debt payment",
    "Strong jobs report signals economic resilience",
    "Oil prices surge on supply concerns",
    "Crypto market crashes 30% in single day",
]

# Analyze each headline
print("Sentiment Analysis Results:")
print("=" * 80)
for headline in headlines:
    result = analyzer.analyze(headline)
    sentiment_emoji = "" if result.score > 0.2 else ("" if result.score < -0.2 else "")
    print(f"{sentiment_emoji} [{result.signal.name:8}] Score: {result.score:+.2f} | {headline[:50]}...")

## 2. Sentiment Filter

The SentimentFilter blocks trades when sentiment is unfavorable:
- Blocks long trades when sentiment is negative
- Blocks short trades when sentiment is positive
- Uses confidence thresholds for decision quality

In [ ]:
# Create sentiment filter
sentiment_filter = create_sentiment_filter(
    min_sentiment_for_long=0.0,  # Require positive sentiment for longs
    max_sentiment_for_short=0.0,  # Require negative sentiment for shorts
    min_confidence=0.5,  # Minimum confidence threshold
    lookback_hours=24,  # Look at last 24 hours of signals
)

print("Sentiment Filter created:")
print(f"  - Min sentiment for long: {sentiment_filter.min_sentiment_for_long}")
print(f"  - Max sentiment for short: {sentiment_filter.max_sentiment_for_short}")
print(f"  - Min confidence: {sentiment_filter.min_confidence}")

In [ ]:
# Simulate adding sentiment signals
symbols = ["AAPL", "TSLA", "MSFT"]
sentiments = {
    "AAPL": [(0.5, 0.8), (0.3, 0.7), (0.4, 0.9)],  # Positive sentiment
    "TSLA": [(-0.3, 0.7), (-0.5, 0.8), (-0.2, 0.6)],  # Negative sentiment
    "MSFT": [(0.1, 0.5), (-0.1, 0.4), (0.05, 0.6)],  # Neutral sentiment
}

# Add signals for each symbol
for symbol, signals in sentiments.items():
    for score, confidence in signals:
        signal = SentimentSignal(
            symbol=symbol,
            score=score,
            confidence=confidence,
            source="test",
            timestamp=datetime.now(),
        )
        sentiment_filter.add_signal(signal)

print("Added sentiment signals for testing")

In [ ]:
# Test entry filtering
test_cases = [
    ("AAPL", "long"),  # Should ALLOW (positive sentiment)
    ("AAPL", "short"),  # Should BLOCK (positive sentiment for short)
    ("TSLA", "long"),  # Should BLOCK (negative sentiment for long)
    ("TSLA", "short"),  # Should ALLOW (negative sentiment)
    ("MSFT", "long"),  # Borderline - check decision
    ("UNKNOWN", "long"),  # No signals - should handle gracefully
]

print("\nSentiment Filter Entry Checks:")
print("=" * 80)
for symbol, direction in test_cases:
    result = sentiment_filter.check_entry(symbol, direction)
    decision_emoji = "" if result.decision == FilterDecision.ALLOW else ""
    print(f"{decision_emoji} {symbol:6} {direction:5} -> {result.decision.name:6} | "
          f"Score: {result.sentiment_score:+.2f}, Conf: {result.confidence:.2f}")
    if result.reason:
        print(f"    Reason: {result.reason.name}")

## 3. News Alert Manager

Detects critical news events that should trigger trading actions:
- Earnings announcements
- FDA approvals
- Regulatory actions
- Mergers and acquisitions

In [ ]:
# Create news alert manager
alert_config = NewsAlertConfig(
    high_impact_sentiment_threshold=0.6,
    circuit_breaker_sentiment_threshold=0.8,
    enable_circuit_breaker_triggers=True,
)

news_manager = NewsAlertManager(config=alert_config)

print("News Alert Manager created:")
print(f"  - High impact threshold: {alert_config.high_impact_sentiment_threshold}")
print(f"  - Circuit breaker threshold: {alert_config.circuit_breaker_sentiment_threshold}")

In [ ]:
# Create and process news events
sample_events = [
    NewsEvent(
        symbol="AAPL",
        headline="Apple beats Q4 earnings expectations",
        event_type=NewsEventType.EARNINGS,
        sentiment_score=0.7,
        timestamp=datetime.now(),
    ),
    NewsEvent(
        symbol="XYZ",
        headline="Company files for bankruptcy protection",
        event_type=NewsEventType.REGULATORY,
        sentiment_score=-0.9,
        timestamp=datetime.now(),
    ),
]

print("\nProcessing News Events:")
print("=" * 80)
for event in sample_events:
    impact = news_manager.assess_impact(event)
    print(f"Symbol: {event.symbol}")
    print(f"  Event: {event.event_type.name}")
    print(f"  Impact: {impact.name}")
    print(f"  Sentiment: {event.sentiment_score:+.2f}")
    print()

## 4. LLM Guardrails

Safety layer that validates all trading decisions:
- Input validation (prevents prompt injection)
- Position size limits
- Confidence-based sizing
- Blocked symbols list
- Trade frequency limits

In [ ]:
# Create trading constraints
constraints = TradingConstraints(
    max_position_size_pct=0.25,  # Max 25% of portfolio per position
    min_confidence_for_trade=0.5,  # Require 50% confidence
    blocked_symbols=["GME", "AMC"],  # Block meme stocks
    max_daily_trades=50,
)

guardrails = create_llm_guardrails(constraints=constraints)

print("LLM Guardrails created:")
print(f"  - Max position size: {constraints.max_position_size_pct:.0%}")
print(f"  - Min confidence: {constraints.min_confidence_for_trade}")
print(f"  - Blocked symbols: {constraints.blocked_symbols}")
print(f"  - Max daily trades: {constraints.max_daily_trades}")

In [ ]:
# Test guardrail validation
trade_decisions = [
    {"action": "buy", "symbol": "AAPL", "position_size": 0.20, "confidence": 0.8, "sentiment_score": 0.5},
    {"action": "buy", "symbol": "AAPL", "position_size": 0.30, "confidence": 0.8, "sentiment_score": 0.5},  # Too large
    {"action": "buy", "symbol": "GME", "position_size": 0.10, "confidence": 0.9, "sentiment_score": 0.3},  # Blocked
    {"action": "buy", "symbol": "TSLA", "position_size": 0.15, "confidence": 0.3, "sentiment_score": 0.1},  # Low confidence
]

print("\nGuardrail Validation Results:")
print("=" * 80)
for trade in trade_decisions:
    result = guardrails.validate_trade_decision(**trade)
    status = "" if result.passed else ""
    print(f"{status} {trade['symbol']:6} size={trade['position_size']:.0%} conf={trade['confidence']:.1f}")
    if not result.passed and result.violations:
        for v in result.violations:
            print(f"    Violation: {v.rule} - {v.message}")

## 5. Confidence-Based Position Sizing

The guardrails can adjust position size based on LLM confidence:
- High confidence (>80%) -> Full position size
- Medium confidence (40-80%) -> Scaled position
- Low confidence (<40%) -> Minimum position size

In [ ]:
# Demonstrate confidence-based sizing
base_size = 0.20  # 20% base position
confidence_levels = [0.3, 0.5, 0.7, 0.9]

print("\nConfidence-Based Position Sizing:")
print("=" * 60)
print(f"Base position size: {base_size:.0%}")
print()

# Scaling config (matches settings.json)
low_threshold = 0.4
high_threshold = 0.8
low_multiplier = 0.5
high_multiplier = 1.0

for conf in confidence_levels:
    if conf < low_threshold:
        multiplier = low_multiplier
    elif conf > high_threshold:
        multiplier = high_multiplier
    else:
        ratio = (conf - low_threshold) / (high_threshold - low_threshold)
        multiplier = low_multiplier + ratio * (high_multiplier - low_multiplier)
    
    adjusted_size = base_size * multiplier
    bar = "" * int(adjusted_size * 20)
    print(f"Confidence {conf:.0%}: {adjusted_size:.1%} {bar}")

## 6. Sentiment History Visualization

Track and visualize sentiment over time to identify trends and potential trading opportunities.

In [ ]:
# Generate synthetic sentiment history
np.random.seed(42)

# 30 days of sentiment data
dates = pd.date_range(start='2025-11-01', periods=30, freq='D')

sentiment_history = pd.DataFrame({
    'Date': dates,
    'AAPL': np.cumsum(np.random.randn(30) * 0.1) * 0.3 + 0.2,  # Trending positive
    'TSLA': np.cumsum(np.random.randn(30) * 0.15) * 0.3 - 0.1,  # More volatile
    'MSFT': np.random.randn(30) * 0.2,  # Neutral/random
}).set_index('Date')

# Clip to [-1, 1] range
sentiment_history = sentiment_history.clip(-1, 1)

sentiment_history.tail(10)

In [ ]:
# Visualize sentiment history
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Sentiment over time
for col in sentiment_history.columns:
    axes[0].plot(sentiment_history.index, sentiment_history[col], label=col, marker='o', markersize=3)

axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].axhline(y=0.3, color='green', linestyle=':', alpha=0.5, label='Long threshold')
axes[0].axhline(y=-0.3, color='red', linestyle=':', alpha=0.5, label='Short threshold')
axes[0].fill_between(sentiment_history.index, -0.3, 0.3, alpha=0.1, color='gray')
axes[0].set_title('Sentiment History by Symbol')
axes[0].set_ylabel('Sentiment Score')
axes[0].set_ylim(-1, 1)
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# Rolling average
rolling_avg = sentiment_history.rolling(window=5).mean()
for col in rolling_avg.columns:
    axes[1].plot(rolling_avg.index, rolling_avg[col], label=f'{col} (5d MA)')

axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_title('5-Day Rolling Average Sentiment')
axes[1].set_ylabel('Sentiment Score')
axes[1].set_xlabel('Date')
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Sentiment-Price Correlation Analysis

Analyze the relationship between sentiment and subsequent price movements.

In [ ]:
# Generate synthetic price data correlated with sentiment
np.random.seed(42)

# Create price changes partially correlated with sentiment
noise = np.random.randn(30) * 0.02
sentiment_effect = sentiment_history['AAPL'].values * 0.01
price_changes = noise + sentiment_effect + 0.0005  # Small positive drift

# Calculate cumulative prices
initial_price = 175.0
prices = initial_price * np.cumprod(1 + price_changes)

# Create analysis dataframe
analysis_df = pd.DataFrame({
    'Date': sentiment_history.index,
    'Sentiment': sentiment_history['AAPL'].values,
    'Price': prices,
    'Return': price_changes,
}).set_index('Date')

# Calculate correlation
correlation = analysis_df['Sentiment'].corr(analysis_df['Return'])
print(f"Sentiment-Return Correlation: {correlation:.3f}")

In [ ]:
# Visualize sentiment-price relationship
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price and sentiment on dual axis
ax1 = axes[0, 0]
ax2 = ax1.twinx()
ax1.plot(analysis_df.index, analysis_df['Price'], 'b-', label='Price', linewidth=2)
ax2.plot(analysis_df.index, analysis_df['Sentiment'], 'r-', label='Sentiment', alpha=0.7)
ax1.set_ylabel('Price ($)', color='blue')
ax2.set_ylabel('Sentiment', color='red')
ax1.set_title('Price vs Sentiment Over Time')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Scatter plot
axes[0, 1].scatter(analysis_df['Sentiment'], analysis_df['Return'], alpha=0.7)
axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].set_xlabel('Sentiment')
axes[0, 1].set_ylabel('Daily Return')
axes[0, 1].set_title(f'Sentiment vs Return (r={correlation:.3f})')

# Sentiment distribution
axes[1, 0].hist(analysis_df['Sentiment'], bins=15, alpha=0.7, edgecolor='black')
axes[1, 0].axvline(x=analysis_df['Sentiment'].mean(), color='red', linestyle='--', 
                   label=f"Mean: {analysis_df['Sentiment'].mean():.3f}")
axes[1, 0].set_xlabel('Sentiment Score')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Sentiment Distribution')
axes[1, 0].legend()

# Return distribution by sentiment bucket
sentiment_buckets = pd.cut(analysis_df['Sentiment'], bins=[-1, -0.2, 0.2, 1], 
                           labels=['Negative', 'Neutral', 'Positive'])
returns_by_sentiment = analysis_df.groupby(sentiment_buckets)['Return'].mean() * 100

colors = ['red', 'gray', 'green']
axes[1, 1].bar(returns_by_sentiment.index, returns_by_sentiment.values, color=colors, alpha=0.7)
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 1].set_xlabel('Sentiment Bucket')
axes[1, 1].set_ylabel('Average Daily Return (%)')
axes[1, 1].set_title('Average Return by Sentiment')

plt.tight_layout()
plt.show()

## 8. Integration Example

Complete example of how sentiment integrates with trading decisions.

In [ ]:
def make_trading_decision(
    symbol: str,
    direction: str,
    base_size: float,
    confidence: float,
    sentiment_filter: SentimentFilter,
    guardrails: LLMGuardrails,
) -> dict:
    """
    Make a complete trading decision with sentiment integration.
    
    Returns decision dict with all validation steps.
    """
    decision = {
        'symbol': symbol,
        'direction': direction,
        'base_size': base_size,
        'confidence': confidence,
        'approved': False,
        'final_size': 0.0,
        'rejection_reasons': [],
    }
    
    # Step 1: Check sentiment filter
    filter_result = sentiment_filter.check_entry(symbol, direction)
    decision['sentiment_score'] = filter_result.sentiment_score
    decision['sentiment_confidence'] = filter_result.confidence
    
    if filter_result.decision == FilterDecision.BLOCK:
        decision['rejection_reasons'].append(f"Sentiment filter: {filter_result.reason.name}")
        return decision
    
    # Step 2: Check guardrails
    action = 'buy' if direction == 'long' else 'sell'
    guardrail_result = guardrails.validate_trade_decision(
        action=action,
        symbol=symbol,
        position_size=base_size,
        confidence=confidence,
        sentiment_score=filter_result.sentiment_score,
    )
    
    if not guardrail_result.passed:
        for v in guardrail_result.violations:
            decision['rejection_reasons'].append(f"Guardrail: {v.message}")
        return decision
    
    # Step 3: Adjust size based on confidence
    low_threshold, high_threshold = 0.4, 0.8
    if confidence < low_threshold:
        multiplier = 0.5
    elif confidence > high_threshold:
        multiplier = 1.0
    else:
        ratio = (confidence - low_threshold) / (high_threshold - low_threshold)
        multiplier = 0.5 + ratio * 0.5
    
    decision['final_size'] = base_size * multiplier
    decision['approved'] = True
    
    return decision

# Test with various scenarios
test_trades = [
    ('AAPL', 'long', 0.15, 0.85),  # Should approve
    ('TSLA', 'long', 0.15, 0.75),  # Blocked by negative sentiment
    ('GME', 'long', 0.10, 0.90),   # Blocked symbol
    ('MSFT', 'long', 0.20, 0.55),  # Borderline confidence
]

print("\nComplete Trading Decision Pipeline:")
print("=" * 90)
for symbol, direction, size, conf in test_trades:
    result = make_trading_decision(
        symbol, direction, size, conf,
        sentiment_filter, guardrails
    )
    
    status = "" if result['approved'] else ""
    print(f"\n{status} {symbol} {direction.upper()} @ {size:.0%}")
    print(f"   Confidence: {conf:.0%} | Sentiment: {result.get('sentiment_score', 0):+.2f}")
    
    if result['approved']:
        print(f"   Final Size: {result['final_size']:.1%}")
    else:
        for reason in result['rejection_reasons']:
            print(f"   Rejection: {reason}")

## Summary

This notebook demonstrated the key components of UPGRADE-014 LLM Sentiment Integration:

1. **LLM Ensemble**: Multi-provider sentiment analysis with dynamic weighting
2. **Sentiment Filter**: Entry filtering based on sentiment direction and confidence
3. **News Alert Manager**: Critical news event detection and impact assessment
4. **LLM Guardrails**: Safety validation for all trading decisions
5. **Confidence Scaling**: Position sizing based on prediction confidence
6. **Persistence**: Storing sentiment history for analysis and feedback

### Next Steps

1. Configure API keys for full ensemble (FinBERT + GPT-4o + Claude)
2. Integrate with live news feeds
3. Backtest sentiment strategies
4. Monitor dynamic weight adjustments
5. Analyze prediction accuracy for feedback loop